In [1]:
#importing all the things

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import ast
import seaborn as sns
sns.set()

%matplotlib inline

In [2]:
#initializing constants
START_DATE = '12/24/2016'
NUM_OF_WEEKS = 4

In [3]:
def get_file_names(start,number): #generates weeknums
    files = []
    ds = dt.datetime.strptime(start,"%m/%d/%Y")
    file_name = (ds.strftime("%y%m%d"))
    files.append(int(file_name))
    for i in range(1,number+1):
        ds = ds+ dt.timedelta(days=7)
        file_name = int(ds.strftime("%y%m%d"))
        files.append(file_name)
    print (files)
    return files


def get_data_parse_dt(week_nums): #imports files from mta website
    url = "turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

def sortLinename(string):
    listLines = ''.join(sorted(string))
    return listLines

In [4]:
data = get_data_parse_dt(get_file_names(START_DATE,NUM_OF_WEEKS))
data.head()

[161224, 161231, 170107, 170114, 170121]


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,03:00:00,REGULAR,5967477,2022101
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,07:00:00,REGULAR,5967485,2022116
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,11:00:00,REGULAR,5967553,2022233
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,15:00:00,REGULAR,5967790,2022331
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,19:00:00,REGULAR,5968186,2022421


In [5]:
data.columns = [column.strip().lower() for column in data.columns] #formatting
data["date_time"] = pd.to_datetime(data.date + " " + data.time, format="%m/%d/%Y %H:%M:%S") #creating datetime column

In [6]:
data.linename = data.linename.apply(sortLinename) #standardizing linenames

In [7]:
data.date = pd.to_datetime(data.date)

In [17]:
oldline = ((data.station=='86 ST') & (data.linename=='456')) | ((data.station=='96 ST') & (data.linename=='6'))
#print (data[oldline].head())
newline = ((data.station=='86 ST-2 AVE') & (data.linename=='Q')) | ((data.station=='96 ST-2 AVE') & (data.linename=='Q'))
#print (data[newline].head())
control = (((data.station=='103 ST') | (data.station=='110 ST')) & (data.linename=='6'))
#print (data[control].head())

mask = ((oldline | newline | control))
ourstations = (data[mask]).copy()

In [18]:
(ourstations
 .groupby(["c/a", "unit", "scp", "station", "date_time", "linename"])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head(5)

,c/a,unit,scp,station,date_time,linename,entries
0,N701,R571,00-00-00,86 ST-2 AVE,2016-12-31 00:00:00,Q,1
8378,R250,R179,00-00-0A,86 ST,2017-01-02 11:00:00,456,1
8356,R250,R179,00-00-0A,86 ST,2016-12-29 19:00:00,456,1
8357,R250,R179,00-00-0A,86 ST,2016-12-29 23:00:00,456,1
8358,R250,R179,00-00-0A,86 ST,2016-12-30 03:00:00,456,1


In [ ]:
#ourstations = data.drop(["exits", "desc"], axis=1, errors="ignore")

In [19]:
ourstations["date"] = pd.to_datetime(ourstations["date"])

In [20]:
ourstations.sort_values(["c/a", "unit", "scp", "station", "date_time"],
                         inplace=True, ascending=False)

In [21]:
dailyData = (ourstations.groupby(["c/a", "unit", "scp", "station", "date", "linename"],as_index=False)
                        .entries.first())

In [22]:
dailyData[["prev_date", "prev_entries"]] = (dailyData.groupby(["c/a", "unit", "scp", "station", "linename"])
                                            ["date", "entries"].transform(lambda grp: grp.shift(1)))
dailyData.dropna(subset=["prev_date"], axis=0, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'date'

In [25]:
dailyData.head()

,c/a,unit,scp,station,date,linename,entries,prev_date,prev_entries
1,N701,R571,00-00-00,86 ST-2 AVE,2017-01-01,Q,2103,2016-12-31,32.0
2,N701,R571,00-00-00,86 ST-2 AVE,2017-01-02,Q,4850,2017-01-01,2103.0
3,N701,R571,00-00-00,86 ST-2 AVE,2017-01-03,Q,8294,2017-01-02,4850.0
4,N701,R571,00-00-00,86 ST-2 AVE,2017-01-04,Q,12193,2017-01-03,8294.0
5,N701,R571,00-00-00,86 ST-2 AVE,2017-01-05,Q,15865,2017-01-04,12193.0


In [26]:
dailyData[dailyData["entries"] < dailyData["prev_entries"]]

,c/a,unit,scp,station,date,linename,entries,prev_date,prev_entries


In [28]:
(dailyData
 .groupby(["c/a", "unit", "scp", "station"])['entries'].count()
    .reset_index().sort_values('entries',ascending=True))

,c/a,unit,scp,station,entries
0,N701,R571,00-00-00,86 ST-2 AVE,20
30,N702A,R572,01-03-04,96 ST-2 AVE,20
29,N702A,R572,01-03-03,96 ST-2 AVE,20
28,N702A,R572,01-03-02,96 ST-2 AVE,20
27,N702A,R572,01-03-01,96 ST-2 AVE,20
26,N702A,R572,01-03-00,96 ST-2 AVE,20
25,N702A,R572,01-00-04,96 ST-2 AVE,20
23,N702A,R572,01-00-02,96 ST-2 AVE,20
22,N702A,R572,01-00-01,96 ST-2 AVE,20
21,N702A,R572,01-00-00,96 ST-2 AVE,20


In [33]:
(dailyData
 .groupby(["c/a", "unit", "scp", "station"])['date'].count()
    .reset_index()['date'].value_counts())

34    40
20    31
Name: date, dtype: int64

In [34]:
(dailyData[dailyData["entries"] < dailyData["prev_entries"]]
    .groupby(["c/a", "unit", "scp", "station"])
    .size())

Series([], dtype: int64)

In [35]:
def get_daily_counts(row, max_counter):
    counter = abs(row["entries"] - row["prev_entries"])
    if counter > max_counter:
        print(row["entries"], row["prev_entries"])
        counter = min(row["entries"], row["prev_entries"])
    if counter > max_counter:
        return 0
    return counter

In [36]:
dailyData["daily_entries"] = dailyData.apply(get_daily_counts, axis=1, max_counter=20000)

In [37]:
dailyData.head()

,c/a,unit,scp,station,date,linename,entries,prev_date,prev_entries,daily_entries
1,N701,R571,00-00-00,86 ST-2 AVE,2017-01-01,Q,2103,2016-12-31,32.0,2071.0
2,N701,R571,00-00-00,86 ST-2 AVE,2017-01-02,Q,4850,2017-01-01,2103.0,2747.0
3,N701,R571,00-00-00,86 ST-2 AVE,2017-01-03,Q,8294,2017-01-02,4850.0,3444.0
4,N701,R571,00-00-00,86 ST-2 AVE,2017-01-04,Q,12193,2017-01-03,8294.0,3899.0
5,N701,R571,00-00-00,86 ST-2 AVE,2017-01-05,Q,15865,2017-01-04,12193.0,3672.0


In [40]:
(dailyData
    .groupby(["station", "date", "linename"])['daily_entries'].sum().reset_index())

,station,date,linename,daily_entries
0,103 ST,2016-12-18,6,7588.0
1,103 ST,2016-12-19,6,15581.0
2,103 ST,2016-12-20,6,16375.0
3,103 ST,2016-12-21,6,16087.0
4,103 ST,2016-12-22,6,15741.0
5,103 ST,2016-12-23,6,14726.0
6,103 ST,2016-12-24,6,8230.0
7,103 ST,2016-12-25,6,5950.0
8,103 ST,2016-12-26,6,7937.0
9,103 ST,2016-12-27,6,11986.0
